In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup_backend import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend 

In [5]:
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend()

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['x', 'sx', 'cz', 'id', 'reset', 'measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155

### Prepare parameters and load dataset

In [6]:
n_qubits = 4

# Load parameters
with open(f"results/get_param/{n_qubits}Q_param.json", "r") as f:
    param = json.load(f)
C = param["C"]
K = param["K"]
n_features = (
    K + 1
)  # 実数部分の特徴量の数。1 は t=0, つまり定数項。虚数も含んだ特徴量の数は 2K+1

print(f"C: {C}")
print(f"K: {K}")

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q_optimal_n_step.json", orient="records"
)
times = df_n_step["t"].values
print(f"times: {times}")
display(df_n_step)

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q_dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

C: 9
K: 8
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268 ]


,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.974201
2,0.698132,2,0.950947
3,1.047198,2,0.946502
4,1.396263,3,0.923480
5,1.745329,3,0.911794
6,2.094395,4,0.892342
7,2.443461,5,0.876343
8,2.792527,5,0.860245


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899247,13.899220,2.726160e-05
1,1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.586491,28.585213,1.278298e-03
2,2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653078,2.653082,3.712600e-06
3,3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222994,7.222891,1.023508e-04
4,4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865096,0.865078,1.800550e-05
5,5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138947,1.138950,2.786300e-06
6,6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060702,9.060659,4.298700e-05
7,7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.366950,4.367027,7.663200e-05
8,8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566096,4.566077,1.851980e-05
9,9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720388,3.720380,8.204800e-06


# Calculate Fourier feature

## Trotter simulation (Simulator, Noise free)

In [7]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": df_n_step["n_step"].values,
    "backend": backend_sim_noisy,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [8]:
_, jobs, _ = run_job(config)

Preparing circuits for sample 0/55
Preparing circuits for sample 1/55
Preparing circuits for sample 2/55
Preparing circuits for sample 3/55
Preparing circuits for sample 4/55
Preparing circuits for sample 5/55
Preparing circuits for sample 6/55
Preparing circuits for sample 7/55
Preparing circuits for sample 8/55
Preparing circuits for sample 9/55
Preparing circuits for sample 10/55
Preparing circuits for sample 11/55
Preparing circuits for sample 12/55
Preparing circuits for sample 13/55
Preparing circuits for sample 14/55
Preparing circuits for sample 15/55
Preparing circuits for sample 16/55
Preparing circuits for sample 17/55
Preparing circuits for sample 18/55
Preparing circuits for sample 19/55
Preparing circuits for sample 20/55
Preparing circuits for sample 21/55
Preparing circuits for sample 22/55
Preparing circuits for sample 23/55
Preparing circuits for sample 24/55
Preparing circuits for sample 25/55
Preparing circuits for sample 26/55
Preparing circuits for sample 27/55
Pr

In [9]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,0.975586,0.672334,0.363052,0.152613,0.331650,-0.067192,0.013495,-0.078342,-0.046889,-0.211450,0.184338,-0.548970,0.158062,-0.643078,-0.094791,-0.390359,-0.317843,13.899247
1,1,0.958008,0.526309,0.477083,-0.130112,0.278486,-0.218649,-0.133844,0.145165,-0.264496,0.350359,-0.080568,0.294208,0.022250,0.163874,-0.055679,0.019447,0.014267,28.586491
2,2,0.963867,0.727220,-0.231140,0.254204,-0.332254,-0.062123,-0.224081,-0.044212,-0.205497,0.159152,-0.300211,0.263787,-0.277621,0.155401,-0.295643,-0.186025,-0.162645,2.653078
3,3,0.959961,0.707394,0.303596,0.225851,0.228938,-0.210169,-0.082334,-0.351267,-0.464830,-0.158354,-0.729921,0.172550,-0.625971,0.292789,-0.227017,0.185062,0.104381,7.222994
4,4,0.969727,0.885708,-0.141601,0.786413,-0.265211,0.664908,-0.383313,0.517898,-0.461055,0.330918,-0.509724,0.253789,-0.520251,0.133125,-0.452086,0.121826,-0.366333,0.865096
5,5,0.962891,0.617733,-0.357973,0.063170,-0.209084,-0.058118,0.101484,-0.052294,0.056955,-0.254737,-0.114542,-0.445206,-0.032036,-0.279682,0.155431,0.160314,-0.049794,1.138947
6,6,0.968750,0.777226,0.228065,0.363492,0.183855,0.152619,0.058405,0.158987,-0.012090,0.117030,0.199686,-0.113366,0.423072,-0.415101,0.364128,-0.525594,0.184646,9.060702
7,7,0.965820,0.761536,-0.144656,0.446063,-0.247140,0.163801,-0.344397,0.105204,-0.257841,0.263893,-0.019280,0.331237,0.256985,0.326076,0.312632,0.269318,0.135064,4.366950
8,8,0.967773,0.792908,0.343220,0.432507,0.484348,0.037200,0.451633,-0.210885,0.285115,-0.273399,-0.038398,-0.078863,-0.288605,0.228388,-0.393014,0.522527,-0.372776,4.566096
9,9,0.967773,0.710661,0.060991,0.260246,0.169447,-0.035351,0.277814,0.029602,0.255848,0.208483,0.050133,0.307797,-0.249113,0.277973,-0.219582,0.166260,-0.072192,3.720388
